In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import random
import os

In [3]:
train=pd.read_csv('spaceship-titanic/train.csv')
test=pd.read_csv('spaceship-titanic/test.csv')
pID=train['PassengerId']

In [4]:
cats=['HomePlanet','Cabin','Destination','CryoSleep','VIP','Name']

In [5]:
def show_nan(df):
    percent_missing = df.isnull().sum() * 100 / len(df)
    missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})
    missing_value_df.sort_values('percent_missing', inplace=True)
    print(missing_value_df)
show_nan(train)
print()

               column_name  percent_missing
PassengerId    PassengerId         0.000000
Transported    Transported         0.000000
Age                    Age         2.059128
RoomService    RoomService         2.082135
Destination    Destination         2.093639
FoodCourt        FoodCourt         2.105142
Spa                    Spa         2.105142
VRDeck              VRDeck         2.162660
Cabin                Cabin         2.289198
Name                  Name         2.300702
HomePlanet      HomePlanet         2.312205
VIP                    VIP         2.335212
ShoppingMall  ShoppingMall         2.392730
CryoSleep        CryoSleep         2.496261



In [6]:
#RoomService, FoodCourt, ShoppingMall, Spa, VRDeck fillna 0

train['RoomService']=train['RoomService'].fillna(0)
train['FoodCourt']=train['FoodCourt'].fillna(0)
train['ShoppingMall']=train['ShoppingMall'].fillna(0)
train['Spa']=train['Spa'].fillna(0)
train['VRDeck']=train['VRDeck'].fillna(0)

test['RoomService']=test['RoomService'].fillna(0)
test['FoodCourt']=test['FoodCourt'].fillna(0)
test['ShoppingMall']=test['ShoppingMall'].fillna(0)
test['Spa']=test['Spa'].fillna(0)
test['VRDeck']=test['VRDeck'].fillna(0)


for i in train.columns:
    if train[i].isna().sum()>0:

        if i not in cats:
            train[i]=train[i].fillna(train.groupby('Transported')[i].transform('mean'))
for i in test.columns:
    if test[i].isna().sum()>0:

        if i not in cats:
            test[i]=test[i].fillna(test[i].mean())

In [7]:
train['Cabin']=train['Cabin'].fillna(method='ffill')
test['Cabin']=test['Cabin'].fillna(method='ffill')

train['deck'] = train['Cabin'].apply(lambda x: x.split('/')[0])
train['num'] = train['Cabin'].apply(lambda x: x.split('/')[1])
train['side'] = train['Cabin'].apply(lambda x: x.split('/')[2])

test['deck'] = test['Cabin'].apply(lambda x: x.split('/')[0])
test['num'] = test['Cabin'].apply(lambda x: x.split('/')[1])
test['side'] = test['Cabin'].apply(lambda x: x.split('/')[2])



del train['Cabin'],test['Cabin']
cats.remove('Cabin')
cats.append('deck')
cats.append('num')
cats.append('side')

In [8]:
train['CryoSleep']=train['CryoSleep'].fillna(False)
test['CryoSleep']=test['CryoSleep'].fillna(False)


for i in test.columns:
    if test[i].isna().sum()>0:
        if i in cats:
            test[i]=test[i].fillna(test[i].value_counts(ascending=True).index[-1])

cats.remove('Name')            

In [9]:
train['group']=train['PassengerId'].apply(lambda x: x.split('_')[0])
test['group']=test['PassengerId'].apply(lambda x: x.split('_')[0])

In [10]:
train['Name']=train['Name'].fillna(method='ffill')
test['Name']=test['Name'].fillna(method='ffill')
temp=pd.DataFrame(train.groupby(['group'])['Name'])
d={}
for i in range(len(temp)):#len(temp)
    past_last_names=[]
    names=list(temp[1][i])
    rltvs=1
    for j in range(len(list(temp[1][i]))):#len(list(temp[1][i]))
        if(names[j].split(' ')[1] in past_last_names):
            rltvs+=1
        past_last_names.append(names[j].split(' ')[1])

    d[f"{temp[0][i]}"]=rltvs

train['has_relatives']=train['group'].map(d)


temp=pd.DataFrame(test.groupby(['group'])['Name'])
d={}
for i in range(len(temp)):#len(temp)
    past_last_names=[]
    names=list(temp[1][i])
    rltvs=1
    for j in range(len(list(temp[1][i]))):#len(list(temp[1][i]))
        if(names[j].split(' ')[1] in past_last_names):
            rltvs+=1
        past_last_names.append(names[j].split(' ')[1])
    d[f"{temp[0][i]}"]=rltvs

test['has_relatives']=test['group'].map(d)
print(train)
del train['Name'],train['group']
del test['Name'],test['group']

     PassengerId HomePlanet  CryoSleep    Destination   Age    VIP  \
0        0001_01     Europa      False    TRAPPIST-1e  39.0  False   
1        0002_01      Earth      False    TRAPPIST-1e  24.0  False   
2        0003_01     Europa      False    TRAPPIST-1e  58.0   True   
3        0003_02     Europa      False    TRAPPIST-1e  33.0  False   
4        0004_01      Earth      False    TRAPPIST-1e  16.0  False   
...          ...        ...        ...            ...   ...    ...   
8688     9276_01     Europa      False    55 Cancri e  41.0   True   
8689     9278_01      Earth       True  PSO J318.5-22  18.0  False   
8690     9279_01      Earth      False    TRAPPIST-1e  26.0  False   
8691     9280_01     Europa      False    55 Cancri e  32.0  False   
8692     9280_02     Europa      False    TRAPPIST-1e  44.0  False   

      RoomService  FoodCourt  ShoppingMall     Spa  VRDeck               Name  \
0             0.0        0.0           0.0     0.0     0.0    Maham Ofracculy 

In [11]:
train['ttl_spnd']=train['RoomService']+train['FoodCourt']+train['ShoppingMall']+train['Spa']+train['VRDeck']
test['ttl_spnd']=test['RoomService']+test['FoodCourt']+test['ShoppingMall']+test['Spa']+test['VRDeck']


In [12]:
train['Adult']=True
train.loc[train['Age']<18, 'Adult']=False
test['Adult']=True
test.loc[test['Age']<18, 'Adult']=False

In [13]:
from sklearn.preprocessing import LabelEncoder
#cats=['HomePlanet','deck','num','side','Destination','CryoSleep','VIP']
#print(train['CryoSleep'])
for i in cats:
    print(i)
    le=LabelEncoder()
    arr=np.concatenate((train[i], test[i])).astype(str)
    le.fit(arr)
    train[i]=le.transform(train[i].astype(str))
    test[i]=le.transform(test[i].astype(str))

t = train['Transported']
del train['Transported']
train['Transported'] = t

HomePlanet
Destination
CryoSleep
VIP
deck
num
side


In [16]:
# seed=1337
# def seed_everything(seed=0):
#     random.seed(seed)
#     os.environ['PYTHONHASHSEED'] = str(seed)
#     np.random.seed(seed)
# seed_everything(seed)
#สุ่มตัวเลข seed=0 เพื่อให้การสุ่มมันตายตัว
y=train['Transported']
y=y.astype(int)
#cats.remove('VIP')
print(train.columns)

from sklearn.model_selection import train_test_split

X=train.drop(columns=['Transported','PassengerId'])

x_tr, x_ts, y_tr, y_ts = train_test_split(X, y, test_size=0.25, random_state=0,shuffle=True)
# else:
#     X_train=train.drop(columns=['Transported','PassengerId'])
#     y_train=y

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP',
       'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'deck',
       'num', 'side', 'has_relatives', 'ttl_spnd', 'Adult', 'Transported'],
      dtype='object')


In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
# Random Forest
top_accuracy = 0
top_value_x = 0
top_value_y = 0
top_model = 0
top_x_values = [] # [7, 4, 6, 11, 12, 19, 20, 22, 29, 30, 47]
top_y_values = [] # [31, 11, 12, 22, 14, 20, 23, 29, 41, 24, 22]

# for value_x in np.arange(1, 100+1, 1):
#     for value_y in np.arange(1, 100+1, 1):
value_x = 69
value_y = 55
value_z = 10

for value_x in np.arange(50, 100+1, 1):
    for value_y in np.arange(50, 100+1, 1):
        for value_z in np.arange(2, 20+1, 1):
            # Random Forest Classifier
            random_forest = RandomForestClassifier(n_estimators=value_x, 
                                                criterion='gini',
                                                max_depth=value_y,
                                                min_samples_split=value_z)

            random_forest.fit(x_tr, y_tr)
            y_pred = random_forest.predict(x_ts)
            random_forest.score(x_tr, y_tr)   
            acc_random_forest = round(metrics.accuracy_score(y_pred, y_ts) * 100, 2)
            if acc_random_forest > top_accuracy:
                for i in range(50):
                    random_forest = RandomForestClassifier(n_estimators=value_x, 
                                                criterion='gini',
                                                max_depth=value_y,
                                                min_samples_split=value_z)
                    random_forest.fit(x_tr, y_tr)
                    y_pred = random_forest.predict(x_ts)
                    random_forest.score(x_tr, y_tr)   
                    acc_random_forest = round(metrics.accuracy_score(y_pred, y_ts) * 100, 2)
                    if acc_random_forest > top_accuracy:
                        top_accuracy = acc_random_forest
                        top_value_x = value_x
                        top_value_y = value_y
                        top_value_z = value_z
                        # top_x_values.append(top_value_x)
                        # top_y_values.append(top_value_y)
                        top_model = random_forest

            print(f'{value_x = :3d}, {value_y = :3d}, {value_z = :3d}, {acc_random_forest = :2.2f} %, {top_value_x = :2d}, {top_value_y = :2d}, {top_value_z = :2d}, {top_accuracy = :2.2f} %', end='\r')
